### --- Day 19: Linen Layout ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/19

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 19);
WriteLines(inputLines, maxCols: 80);

Loading puzzle file: Day19.txt
Total lines: 402
Max line length: 2902

brbwuur, bwuw, bbb, wbgu, gguw, ubuguw, rrw, brgr, uur, urguwruu, uubr, ru, uuw,

buwugbgrgururgwrgrrugbwgrwurgbubrggruwugwgrwguuurwu
bwbrurbwgurggbbwbrbwubrurrwrwwwruurbrrguuubg
buubbubwwwgugwgwruwbrwbbgrrwwrurrbwgwbbrbugbbubbbwuwubrbg


In [4]:
string[] testInputLines = [
    "r, wr, b, g, bwu, rb, gb, br",
    "",
    "brwrr",
    "bggr",
    "gbbr",
    "rrbgbr",
    "ubwu",
    "bwurrg",
    "brgr",
    "bbrgwb",
];

I reckon there is a more sophisticated and efficient approach to this (perhaps a trie??), but given the number of designs and patterns, the most basic approach to "chomp" off bits of the string seems feasible. We'll start with that and investigate other solutions later.

In [5]:
(string[] patterns, string[] designs) ParseLines(string[] inputLines)
{
    var (patternLine, designLines) = inputLines.SeparateBy(line => line is "").ToArray();

    var patterns = patternLine[0].Split(", ").ToArray();
    return (patterns, designLines);
}

In [6]:
bool CanMake(string[] patterns, string design)
{
    Queue<string> designs = new();
    designs.Enqueue(design);

    // Make sure we don't regenerate the same string out of different pattern
    // combinations
    HashSet<string> seen = new();

    while (designs.TryDequeue(out var currentDesign))
    {
        seen.Add(currentDesign);

        foreach (var pattern in patterns)
        {
            if (currentDesign.StartsWith(pattern))
            {
                var remainDesign = currentDesign.Substring(pattern.Length);
                if (remainDesign is "")
                {
                    return true;
                }
                
                if (seen.Contains(remainDesign))
                {
                    // Already generated this sub-design out of different patterns
                    continue;
                }

                designs.Enqueue(remainDesign);
            }
        }
    }

    return false;
}

In [7]:
int CountMakeable(string[] inputLines)
{
    var (patterns, designs) = ParseLines(inputLines);

    return designs.Where(design => CanMake(patterns, design)).Count();
}

In [8]:
// In this example, 6 of the eight designs are possible with the available towel patterns.

var testAnswer = CountMakeable(testInputLines);
Console.WriteLine(testAnswer);

6


In [9]:
// To get into the onsen as soon as possible, consult your list of towel
// patterns and desired designs carefully. How many designs are possible?

var part1Answer = CountMakeable(inputLines);
Console.WriteLine(part1Answer);

233


In [10]:
// 233 is correct!
Ensure(233, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/19

In summary, for part 2 we need to count all possible ways that the given patterns can make a particular design.

I think we can achieve this with a dynamic programming approach. Starting at the end of the string...

* There is 1 way to make `r`
* There are 2 ways to make `br`:
    * `b` + `r`
    * `br`
* There are 3 ways to make `gbr`
    * `g` + `br` (2 ways)
    * `gb` (1) * `r` (1 way)

Essentially, for a given character position `i`, the value for `patterns(i) = sum(patterns(i + b))`, for each pattern of length `b` that matches the substring starting at `i`.

In [12]:
// Our cache of known combos for a given substring.

// Mapping is: substring length => combo count.
using CountCache = SCG.Dictionary<int, long>;

In [13]:
long CountPatternCombos(string[] stripePatterns, string design)
{
    CountCache countCache = new();

    long CountOne(string[] stripePatterns, ReadOnlySpan<char> designSubstr)
    {
        if (countCache.TryGetValue(designSubstr.Length, out var result))
        {
            return result;
        }
        
        foreach (var stripePattern in stripePatterns)
        {
            if (designSubstr.StartsWith(stripePattern))
            {
                // We can potentially make this design with the current pattern

                var nextSubstr = designSubstr[stripePattern.Length..];
               
                result += nextSubstr.Length switch {
                    // Fully satisfied the design
                    0 => 1,
                    // Still more of the design to go
                    _ => CountOne(stripePatterns, nextSubstr)
                };
            }
        }
        countCache[designSubstr.Length] = result;

        return result;
    }

    return CountOne(stripePatterns, design);
}

In [14]:
long CountAllPatternCombos(string[] inputLines)
{
    var (stripePatterns, designs) = ParseLines(inputLines);

    return designs.Select(design => (long)CountPatternCombos(stripePatterns, design)).Sum();
}

In [15]:
// Adding up all of the ways the towels in this example could be arranged into
// the desired designs yields 16 (2 + 1 + 4 + 6 + 1 + 2).

var part2TestAnswer = CountAllPatternCombos(testInputLines);
Console.WriteLine(part2TestAnswer);

16


In [16]:
// They'll let you into the onsen as soon as you have the list. What do you get
// if you add up the number of different ways you could make each design?

var part2Answer = CountAllPatternCombos(inputLines);
Console.WriteLine(part2Answer);

691316989225259


In [17]:
// 691316989225259 is correct!
Ensure(691316989225259, part2Answer);